In [6]:
import urllib.request

import pandas as pd

In [9]:
# Load dataset from Johns Hopkins website
def load_url_as_df(url):
    """Loads csv data from a remote URL as a pandas Dataframe"""
    
    with urllib.request.urlopen(url) as stream:
        return pd.read_csv(stream, index_col='Country/Region')

In [11]:
df_confirmed = load_url_as_df('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
df_confirmed.head()

,Province/State,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Thailand,NaN,15.0000,101.0000,2,3,5,7,8,8,14,...,48,50,50,50,53,59,70,75,82,114
Japan,NaN,36.0000,138.0000,2,1,2,2,4,4,7,...,420,461,502,511,581,639,639,701,773,839
Singapore,NaN,1.2833,103.8333,0,1,3,3,4,5,7,...,130,138,150,150,160,178,178,200,212,226
Nepal,NaN,28.1667,84.2500,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Malaysia,NaN,2.5000,112.5000,0,0,0,3,4,4,4,...,83,93,99,117,129,149,149,197,238,428
